In [28]:
from collections import Counter
import numpy as np

In [4]:
# Each individual in a generation is represented as the set of ancestors
# from the initial generation. 

N = 10_000
MALE_PROPORTION = 0.5

In [5]:
def make_couples(generation:np.ndarray, male_proportion:float = MALE_PROPORTION) -> np.ndarray:
    """Split the generation into male and female segments, form couples
    
    Returns: array of couple, where each couple is represented as the union of their ancestors
    """
    # population size and male_proportion are parameters to allow for more dynamic simulations
    population_size = len(generation)
    num_males = int(male_proportion * population_size)
    num_females = population_size - num_males
    men = generation[:num_males]
    women = generation[num_males:]
    np.random.shuffle(men)
    np.random.shuffle(women)
    couples = []
    for i in range(min(num_males, num_females)):
        couples.append(men[i].union(women[i]))
    if not num_males == num_females:
        for i in range(abs(num_females - num_males)):
            if num_males < num_females:
                couples.append(men[i].union(women[i + num_males]))
            else:
                couples.append(women[i].union(men[i + num_females]))
    return np.array(couples)


In [6]:
def create_next_generation(couples:np.ndarray, population_size:int = N) -> np.ndarray:
    """Create the next population generation"""
    num_couples = len(couples)
    children_index = np.random.randint(0, high=num_couples, size=population_size)
    return np.take(couples, children_index)


In [7]:
def count_descendants(generation:np.ndarray) -> Counter:
    """Return a Counter with info on how many descendants first-gen ancestors have.
    
    Note: Ancestors with no descendants do not have an indexed entry in the Counter
    """
    c = Counter()
    for individual in generation:
        c += Counter(individual)
    return c

In [8]:
def is_genealogical_ae_present(descendant_counts:Counter , population_size:int = N) -> bool:
    if max(descendant_counts.values()) == population_size:
        return True
    else:
        return False

In [9]:
# In the initial generation, each individual's ancestry is self
generation = np.array([{i} for i in range(N)])
for i in range(1000):
    couples = make_couples(generation)
    next_gen = create_next_generation(couples)
    descendants = count_descendants(next_gen)
    if is_genealogical_ae_present(descendants):
        break
    else:
        generation = next_gen



In [29]:
class SimReport():

    start_population:int
    num_generations:int
    num_ancestors_represented:int
    num_ae_couples:int
    distribution_of_couples_descendants:Counter

report = SimReport()
report.start_population = N
report.num_generations = i + 1 # Python iteration starts at zero, but we start at 1
report.num_ancestors_represented = len(descendants)
d = Counter(descendants.values())
c = Counter()
for k in d:
    c[k] = int(d[k]/2)
report.distribution_of_couples_descendants = c
report.num_ae_couples = report.distribution_of_couples_descendants[N]




In [30]:
print("Initial ancestral couples:", int(report.start_population / 2))
print("Number of generations until A/E couple emerged:", report.num_generations)
print("Number of ancestral couples still in gene pool at simulation end:", int(report.num_ancestors_represented/2))
print("Number of A/E couples at simulation end:", int(report.num_ae_couples))
almost_ae = 0
for k in report.distribution_of_couples_descendants:
    if k >= N * 0.9:
        almost_ae += report.distribution_of_couples_descendants[k]
print("Number of ancestral couples who are ancestors of 90+% of final generation:", almost_ae)

Initial ancestral couples: 5000
Number of generations until A/E couple emerged: 14
Number of ancestral couples still in gene pool at simulation end: 3934
Number of A/E couples at simulation end: 16
Number of ancestral couples who are ancestors of 90+% of final generation: 1416
